# Case DS - Grupo Boticário

# Modelo

## 1. Importação dos pacotes e funções

### 1.1 Bibliotecas utilizadas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, max_error
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_val_predict, ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn import model_selection
from IPython.display import display
import warnings
import datetime


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### 1.2 Configurações

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
sns.set_palette("gist_ncar")
sns.set_style("whitegrid")
seed = 42

In [3]:
def correlated_columns(df, threshold):
    return (
        df.corr()
        .pipe(
            lambda df1: pd.DataFrame(
                np.tril(df1, k=-1),
                columns=df.columns,
                index=df.columns,
            )
        )
        .stack()
        .rename("kendall")
        .pipe(
            lambda s: s[
                s.abs() > threshold
            ].reset_index()
        )
        .query("level_0 not in level_1")
    )

In [4]:
def models_score(models, X_train, y_train, X_test, y_test, scoring = 'neg_mean_absolute_percentage_error', n_splits=10, figsize = (20, 4)):
  results_cv = []
  results = []
  names = []
  best_sens_test = 0
  best_clf = ''
  pd.options.display.float_format = '{:,.2f}'.format
    
  for name, model in models:
    time0 = datetime.datetime.now()
    names.append(name)
    print('Iniciando processamento do modelo '+ name)
    # cross validation
    kfold = model_selection.KFold(n_splits=n_splits)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results_cv.append(cv_results) # acc cv
    # model training
    time1 = datetime.datetime.now()
    model.fit(X_train, y_train)
    time2 = datetime.datetime.now()
    # model predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    # MAE Mean Absolute Error
    mae = mean_absolute_error(y_test,y_pred_test)
    mae_train = mean_absolute_error(y_train,y_pred_train) 
    # MAPE mean_absolute_percentage_error
    mape = mean_absolute_percentage_error(y_test,y_pred_test)
    mape_train = mean_absolute_percentage_error(y_train,y_pred_train)
    # Max Error
    max_erro = max_error(y_test,y_pred_test)
    max_erro_train = max_error(y_train,y_pred_train)
    # R2 r2_score
    r2 = r2_score(y_test,y_pred_test)
    r2_train = r2_score(y_train,y_pred_train)
    # training time
    duration=time2-time1
    # df
    results.append({'Name': name,
                    'CV MAPE mean': np.mean(cv_results),
                    'CV MAPE std': np.std(cv_results),
                    'MAE (train)': mae_train,
                    'MAE (test)': mae,
                    'Max Error (train)' : max_erro,
                    'Max Error (test)'  : max_erro_train,
                    'MAPE (train)' : mape_train,
                    'MAPE (test)' : mape,
                    'R2 (train)' : r2_train,
                    'R2 (test)' : r2,
                    'Training time (s)': duration.total_seconds()})
    time3 = datetime.datetime.now()
    duration2=time3-time0
    print(f'Total training+CV time (s): {duration2}')
  results_df = pd.DataFrame(results).sort_values(by='MAPE (test)').reset_index(drop = True)
  results_cv_df = pd.DataFrame(results_cv).T
  results_cv_df.columns = names
  print('----------------RESULTS-------------- \n')
  print('Best MAPE in train (CV mean): {:0.2f}'.format(np.min(np.mean(results_cv, axis=1))))
  print('Best MAE in test: {:0.2f}'.format(results_df['MAE (test)'].min()))
  print('Best MAPE in test: {:0.2f} \n'.format(results_df['MAPE (test)'].min()))
  print('Best R2 in test: {:0.2f} \n'.format(np.abs(results_df['R2 (test)'].min())))
  print('Best Max_error in test: {:0.2f} \n'.format(np.abs(results_df['Max Error (test)'].min())))
  print('-----------MODELS COMPARISON--------- \n')
  display(results_df)

  return results_cv_df, results_df

In [5]:
def fill_categoric_field_with_value(serie):
    names = serie.unique()
    values = list(range(1, names.size + 1))
    
    #a tabela de valores continha um float(nan) mapeado para um valor inteiro. Solução foi mudar na tabela de valores colocando o None
    nan_index = np.where(pd.isna(names))
    if len(nan_index) > 0 and len(nan_index[0]) > 0:
        nan_index = nan_index[0][0]
        values[nan_index] = None
    #else:
        #print("Não encontrou nan em " + str(names))
        
    return serie.replace(names,values)

## 2. Obtenção dos dados 

## 2.1 Importação dos dados

In [6]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("C:/temp/case_ds/case_ds_gdem.sqlite3")
df = pd.read_sql_query("SELECT * from vendas", con)

print('A base possui', df.shape[0], 'linhas e', df.shape[1], 'variáveis (atributos)')

A base possui 52094 linhas e 14 variáveis (atributos)


In [7]:
df.head() # exibindo as primeiras linhas

,COD_MATERIAL,COD_CICLO,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,FLG_DATA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,FLG_CAMPANHA_MKT_F,PCT_DESCONTO,VL_PRECO,QT_VENDA
0,6246,201908,anon_S12,anon_S53,1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,461.4,240.0
1,6246,201915,anon_S12,anon_S53,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,461.4,240.0
2,6306,201916,anon_S12,anon_S53,1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,521.4,240.0
3,6480,201801,anon_S12,anon_S17,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,833.4,528.0
4,6480,201802,anon_S12,anon_S17,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,833.4,564.0


In [8]:
df.tail() # exibindo as últimas linhas

,COD_MATERIAL,COD_CICLO,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,FLG_DATA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,FLG_CAMPANHA_MKT_F,PCT_DESCONTO,VL_PRECO,QT_VENDA
52089,501084,202017,anon_S2,anon_S24,1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,443.400,NaN
52090,501084,202101,anon_S2,anon_S24,0,0.0,0.5,0.0,0.0,0.0,0.0,20.0,443.400,NaN
52091,502386,202016,anon_S12,anon_S25,1,1.0,1.0,0.0,0.0,0.0,0.0,25.0,647.430,NaN
52092,502386,202017,anon_S12,anon_S25,1,0.0,1.0,0.0,0.0,0.0,0.0,20.0,727.215,NaN
52093,502386,202101,anon_S12,anon_S25,0,0.0,0.5,0.0,0.0,0.0,0.0,20.0,783.450,NaN


In [9]:
print('A base possui', df.shape[0], 'linhas e', df.shape[1], 'variáveis (atributos)')

A base possui 52094 linhas e 14 variáveis (atributos)


# Conclusões da análise exploratoria
### Colunas FLG_CAMPANHA_MKT_E e FLG_CAMPANHA_MKT_F podem ser retiradas
### Colunas todas com 52.087 linhas não nulas, exceto pela coluna de desconto (que será corrigida) e a coluna de QT_Vendas, que estão nulos somente os casos que usaremos para prever.
### Retirar casos onde o VL_PRECO for nulo (somente 6 casos)
### Retirar PCT_Desconto que for maior que 100
### Separar couna COD_CICLO, em ano e ciclo.
### Categorização dos dados das colunas 'str'.
### Criação das colunas DATA_INICIO e DATA_FIM dos ciclos nos anos
### Analisar o que fazer com a PCT_DESCONTO que estiver nulo (34197 casos):
#### - colocar 0.0
#### - colocar a moda
#### - colocar a média


## 3. Ajustes EDA

In [10]:
df['CICLO'] = df['COD_CICLO'].astype(str).str.slice(start=4).astype(int)
df['ANO'] = df['COD_CICLO'].astype(str).str.slice(stop=4).astype(int)

In [11]:
df.drop(['FLG_CAMPANHA_MKT_E' , 'FLG_CAMPANHA_MKT_F'],axis = 1,inplace=True)

In [12]:
df.dropna(subset = ['VL_PRECO'], inplace = True ) 

In [13]:
df = df.drop(df[df['PCT_DESCONTO'] > 100].index)

In [14]:
df['DES_CATEGORIA_MATERIAL'] = fill_categoric_field_with_value(df['DES_CATEGORIA_MATERIAL'])
df['DES_MARCA_MATERIAL'] = fill_categoric_field_with_value(df['DES_MARCA_MATERIAL'])

In [15]:
df['PCT_DESCONTO'].fillna(0, inplace=True)

In [42]:
a = df.groupby('ANO')['CICLO'].idxmax()
df_tamanho_ciclos = df.loc[a][['ANO', 'CICLO']]

#assumption, 2021 tera 17 ciclos
df_tamanho_ciclos['CICLO'] = df_tamanho_ciclos.apply(lambda x : 17 if x['ANO'] == 2021 else x['CICLO'] , axis = 1)
df_tamanho_ciclos['DELTA'] = 365 / df_tamanho_ciclos['CICLO']

# Pegar o dia de inicio de cada ano e somar os dias de cada ciclo
def obter_data(row , df_tamanho_ciclos):
    ciclo = row['CICLO']
    ano = row['ANO']
    delta = df_tamanho_ciclos.loc[df_tamanho_ciclos['ANO'] == ano]['DELTA'].values[0]
    a_data_inicial = datetime.date(int(ano), 1, 1) + datetime.timedelta(days=(delta * (ciclo - 1)) )
    a_data_final = a_data_inicial + datetime.timedelta(days=delta)
    return a_data_inicial , a_data_final
    
df[['DATA_INICIO' , 'DATA_FIM']] = df.apply(obter_data , df_tamanho_ciclos = df_tamanho_ciclos , result_type='expand' , axis=1)
df.sort_values(by='DATA_INICIO',inplace=True)


df['DATA_INICIO'] = df['DATA_INICIO'].apply(lambda x: datetime.datetime.combine(x, datetime.datetime.min.time())).apply(lambda x: x.timestamp())
df['DATA_FIM'] = df['DATA_FIM'].apply(lambda x: datetime.datetime.combine(x, datetime.datetime.min.time())).apply(lambda x: x.timestamp())

## 4. Base treino e teste

## 4.1 Separando a base para a previsão da base de teste/treino

### Base Teste/Treino

In [43]:
df_base=df.dropna(subset=['QT_VENDA'], inplace=False)

In [44]:
df_base.head()

,COD_MATERIAL,COD_CICLO,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,FLG_DATA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,PCT_DESCONTO,VL_PRECO,QT_VENDA,CICLO,ANO,DATA_INICIO,DATA_FIM
18623,153096,201801,1,20,0,0.0,1.0,0.5,0.0,21.640,632.55,442602.0,1,2018,1.514765e+09,1.516493e+09
21620,166746,201801,5,48,0,0.0,1.0,0.5,0.0,30.485,461.79,94608.0,1,2018,1.514765e+09,1.516493e+09
38030,432618,201801,3,54,0,1.0,0.0,0.0,0.0,0.000,461.40,47670.0,1,2018,1.514765e+09,1.516493e+09
8743,124296,201801,2,40,0,0.0,0.0,0.0,0.0,0.000,413.40,240.0,1,2018,1.514765e+09,1.516493e+09
8748,124842,201801,2,40,0,0.0,1.0,0.0,0.0,10.000,365.40,3066.0,1,2018,1.514765e+09,1.516493e+09


In [45]:
print('A base possui', df_base.shape[0], 'linhas e', df_base.shape[1], 'variáveis (atributos)')

A base possui 48939 linhas e 16 variáveis (atributos)


In [46]:
print(df_base.isnull().sum())

COD_MATERIAL              0
COD_CICLO                 0
DES_CATEGORIA_MATERIAL    0
DES_MARCA_MATERIAL        0
FLG_DATA                  0
FLG_CAMPANHA_MKT_A        0
FLG_CAMPANHA_MKT_B        0
FLG_CAMPANHA_MKT_C        0
FLG_CAMPANHA_MKT_D        0
PCT_DESCONTO              0
VL_PRECO                  0
QT_VENDA                  0
CICLO                     0
ANO                       0
DATA_INICIO               0
DATA_FIM                  0
dtype: int64


In [47]:
pd.options.display.max_rows = 118
print('Valores únicos em cada variável: \n')
df_base.nunique(dropna=False).sort_values()

Valores únicos em cada variável: 



FLG_DATA                      2
ANO                           3
FLG_CAMPANHA_MKT_A            4
FLG_CAMPANHA_MKT_C            4
DES_CATEGORIA_MATERIAL        6
FLG_CAMPANHA_MKT_B            7
FLG_CAMPANHA_MKT_D            7
CICLO                        18
COD_CICLO                    50
DATA_INICIO                  50
DATA_FIM                     50
DES_MARCA_MATERIAL           82
PCT_DESCONTO                708
COD_MATERIAL               2176
QT_VENDA                  14266
VL_PRECO                  14706
dtype: int64

### Base para a previsão

In [48]:
df_a_prever = df[df['QT_VENDA'].isnull()]

In [49]:
print('A base possui', df_a_prever.shape[0], 'linhas e', df_a_prever.shape[1], 'variáveis (atributos)')

A base possui 3148 linhas e 16 variáveis (atributos)


In [50]:
print(df_a_prever.isnull().sum())

COD_MATERIAL                 0
COD_CICLO                    0
DES_CATEGORIA_MATERIAL       0
DES_MARCA_MATERIAL           0
FLG_DATA                     0
FLG_CAMPANHA_MKT_A           0
FLG_CAMPANHA_MKT_B           0
FLG_CAMPANHA_MKT_C           0
FLG_CAMPANHA_MKT_D           0
PCT_DESCONTO                 0
VL_PRECO                     0
QT_VENDA                  3148
CICLO                        0
ANO                          0
DATA_INICIO                  0
DATA_FIM                     0
dtype: int64


In [51]:
pd.options.display.max_rows = 118
print('Valores únicos em cada variável: \n')
df_a_prever.nunique(dropna=False).sort_values()

Valores únicos em cada variável: 



QT_VENDA                     1
FLG_DATA                     2
ANO                          2
COD_CICLO                    3
FLG_CAMPANHA_MKT_C           3
CICLO                        3
DATA_INICIO                  3
DATA_FIM                     3
FLG_CAMPANHA_MKT_D           4
FLG_CAMPANHA_MKT_A           5
DES_CATEGORIA_MATERIAL       6
FLG_CAMPANHA_MKT_B           6
DES_MARCA_MATERIAL          51
PCT_DESCONTO               153
COD_MATERIAL              1205
VL_PRECO                  1754
dtype: int64

## 4.2 Separação teste e treino

In [52]:
# divide a base em partições de treino (70%) e teste (30%) com dados estratificados pela variável alvo
X_train, X_test, y_train, y_test = train_test_split(
    df_base.drop(['QT_VENDA'], axis=1), 
    df_base['QT_VENDA'],
    test_size=0.3,
    stratify = df_base[['ANO','CICLO']])

print("Número de registros em X_train: ", X_train.shape)
print("Número de registros em y_train: ", y_train.shape)
print("Número de registros em X_test: ", X_test.shape)
print("Número de registros em y_test: ", y_test.shape)

Número de registros em X_train:  (34257, 15)
Número de registros em y_train:  (34257,)
Número de registros em X_test:  (14682, 15)
Número de registros em y_test:  (14682,)


In [53]:
df_base.groupby(df_base['DES_MARCA_MATERIAL']).count().head(100)

,COD_MATERIAL,COD_CICLO,DES_CATEGORIA_MATERIAL,FLG_DATA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,PCT_DESCONTO,VL_PRECO,QT_VENDA,CICLO,ANO,DATA_INICIO,DATA_FIM
DES_MARCA_MATERIAL,,,,,,,,,,,,,,,
1,203,203,203,203,203,203,203,203,203,203,203,203,203,203,203
2,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204,2204
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
5,214,214,214,214,214,214,214,214,214,214,214,214,214,214,214
6,136,136,136,136,136,136,136,136,136,136,136,136,136,136,136
7,308,308,308,308,308,308,308,308,308,308,308,308,308,308,308
8,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
9,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297


In [54]:
df_X_train=pd.DataFrame(X_train)
df_X_train.nunique(dropna=False).sort_values()

FLG_DATA                      2
FLG_CAMPANHA_MKT_C            3
ANO                           3
FLG_CAMPANHA_MKT_A            4
DES_CATEGORIA_MATERIAL        6
FLG_CAMPANHA_MKT_B            7
FLG_CAMPANHA_MKT_D            7
CICLO                        18
COD_CICLO                    50
DATA_INICIO                  50
DATA_FIM                     50
DES_MARCA_MATERIAL           80
PCT_DESCONTO                611
COD_MATERIAL               2071
VL_PRECO                  11303
dtype: int64

In [55]:
df_X_test=pd.DataFrame(X_test)
df_X_test.nunique(dropna=False).sort_values()

FLG_DATA                     2
FLG_CAMPANHA_MKT_A           3
ANO                          3
FLG_CAMPANHA_MKT_C           4
DES_CATEGORIA_MATERIAL       6
FLG_CAMPANHA_MKT_B           7
FLG_CAMPANHA_MKT_D           7
CICLO                       18
COD_CICLO                   50
DATA_INICIO                 50
DATA_FIM                    50
DES_MARCA_MATERIAL          72
PCT_DESCONTO               435
COD_MATERIAL              1865
VL_PRECO                  5820
dtype: int64

In [56]:
data_train = X_train.join(y_train)

In [57]:
data_test = X_test.join(y_test)

In [58]:
data_train.shape

(34257, 16)

In [59]:
data_train.nunique(dropna=False).sort_values()

FLG_DATA                      2
FLG_CAMPANHA_MKT_C            3
ANO                           3
FLG_CAMPANHA_MKT_A            4
DES_CATEGORIA_MATERIAL        6
FLG_CAMPANHA_MKT_B            7
FLG_CAMPANHA_MKT_D            7
CICLO                        18
COD_CICLO                    50
DATA_INICIO                  50
DATA_FIM                     50
DES_MARCA_MATERIAL           80
PCT_DESCONTO                611
COD_MATERIAL               2071
VL_PRECO                  11303
QT_VENDA                  11585
dtype: int64

In [60]:
data_test.shape

(14682, 16)

In [61]:
data_test.nunique(dropna=False).sort_values()

FLG_DATA                     2
FLG_CAMPANHA_MKT_A           3
ANO                          3
FLG_CAMPANHA_MKT_C           4
DES_CATEGORIA_MATERIAL       6
FLG_CAMPANHA_MKT_B           7
FLG_CAMPANHA_MKT_D           7
CICLO                       18
COD_CICLO                   50
DATA_INICIO                 50
DATA_FIM                    50
DES_MARCA_MATERIAL          72
PCT_DESCONTO               435
COD_MATERIAL              1865
VL_PRECO                  5820
QT_VENDA                  6585
dtype: int64

# 5. Análise de correlação

## 5.1 Correlação com a variável target

In [62]:
df_corr = data_train.corr(method='spearman')['QT_VENDA'][:]
pd.DataFrame(df_corr).sort_values(by="QT_VENDA", ascending = False)

,QT_VENDA
QT_VENDA,1.000000
COD_MATERIAL,0.557657
PCT_DESCONTO,0.555445
FLG_CAMPANHA_MKT_B,0.543563
FLG_CAMPANHA_MKT_D,0.337301
FLG_CAMPANHA_MKT_C,0.222088
DES_MARCA_MATERIAL,0.205133
FLG_CAMPANHA_MKT_A,0.093735
COD_CICLO,0.063959
DATA_INICIO,0.063959


In [63]:
df_corr = data_train.corr(method='pearson')['QT_VENDA'][:]
pd.DataFrame(df_corr).sort_values(by="QT_VENDA", ascending = False)

,QT_VENDA
QT_VENDA,1.000000
FLG_CAMPANHA_MKT_B,0.338974
PCT_DESCONTO,0.328489
FLG_CAMPANHA_MKT_C,0.218376
FLG_CAMPANHA_MKT_D,0.178326
FLG_CAMPANHA_MKT_A,0.122465
COD_MATERIAL,0.120298
VL_PRECO,0.099371
DES_MARCA_MATERIAL,0.021054
CICLO,0.012553


In [64]:
df_corr = data_train.corr(method='kendall')['QT_VENDA'][:]
pd.DataFrame(df_corr).sort_values(by="QT_VENDA", ascending = False)

,QT_VENDA
QT_VENDA,1.000000
FLG_CAMPANHA_MKT_B,0.440080
PCT_DESCONTO,0.432480
COD_MATERIAL,0.387241
FLG_CAMPANHA_MKT_D,0.274943
FLG_CAMPANHA_MKT_C,0.181713
DES_MARCA_MATERIAL,0.145036
FLG_CAMPANHA_MKT_A,0.076867
ANO,0.042492
COD_CICLO,0.042420


## 5.2 Multicolinariedade

In [65]:
df_corr = data_train.corr(method='kendall')
df_corr.head(50)

,COD_MATERIAL,COD_CICLO,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,FLG_DATA,FLG_CAMPANHA_MKT_A,FLG_CAMPANHA_MKT_B,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,PCT_DESCONTO,VL_PRECO,CICLO,ANO,DATA_INICIO,DATA_FIM,QT_VENDA
COD_MATERIAL,1.000000,0.265519,0.108579,0.195830,-0.008736,0.114372,0.237266,0.063199,0.198709,0.242084,0.019893,0.045413,0.289098,0.265519,0.265519,0.387241
COD_CICLO,0.265519,1.000000,0.001059,0.061047,0.023383,0.014646,0.021082,-0.071632,0.073664,0.043142,0.049509,0.249550,0.824139,1.000000,1.000000,0.042420
DES_CATEGORIA_MATERIAL,0.108579,0.001059,1.000000,0.142588,0.001125,0.002384,0.025021,-0.016967,0.009814,0.021867,-0.161385,-0.000744,0.001712,0.001059,0.001059,-0.039471
DES_MARCA_MATERIAL,0.195830,0.061047,0.142588,1.000000,-0.006393,0.031600,0.090317,0.012633,0.161430,0.120362,-0.001355,0.005268,0.069088,0.061047,0.061047,0.145036
FLG_DATA,-0.008736,0.023383,0.001125,-0.006393,1.000000,0.015638,-0.078322,-0.022534,0.014332,-0.058497,0.018754,0.238895,-0.066485,0.023383,0.023383,0.007178
FLG_CAMPANHA_MKT_A,0.114372,0.014646,0.002384,0.031600,0.015638,1.000000,0.094642,0.145763,0.080129,0.085280,-0.002841,0.029930,0.006144,0.014646,0.014646,0.076867
FLG_CAMPANHA_MKT_B,0.237266,0.021082,0.025021,0.090317,-0.078322,0.094642,1.000000,0.342564,0.315560,0.834675,-0.095807,0.009227,0.021898,0.021082,0.021082,0.440080
FLG_CAMPANHA_MKT_C,0.063199,-0.071632,-0.016967,0.012633,-0.022534,0.145763,0.342564,1.000000,0.158838,0.333409,-0.027552,-0.014339,-0.079704,-0.071632,-0.071632,0.181713
FLG_CAMPANHA_MKT_D,0.198709,0.073664,0.009814,0.161430,0.014332,0.080129,0.315560,0.158838,1.000000,0.500184,0.003528,0.069979,0.060497,0.073664,0.073664,0.274943
PCT_DESCONTO,0.242084,0.043142,0.021867,0.120362,-0.058497,0.085280,0.834675,0.333409,0.500184,1.000000,-0.094867,-0.008775,0.054312,0.043142,0.043142,0.432480


# 6. Modelagem Estatística

## 6.1 Seleção de modelos

### 6.1.1 Redes Neurais

In [66]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test =scaler.transform(X_test)

models=[]
models.append(('Rede Neural - MLP5 10',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(10,10,10,10,10), tol = 0.01)))
models.append(('Rede Neural - MLP5 15',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(15,15,15,15,15), tol = 0.01)))
models.append(('Rede Neural - MLP5 25',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(25,25,25,25,25), tol = 0.01)))
models.append(('Rede Neural - MLP5 30',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(30,30,30,30,30), tol = 0.01)))
models.append(('Rede Neural - MLP8 10',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(10,10,10,10,10,10,10,10), tol = 0.01)))
models.append(('Rede Neural - MLP8 15',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(15,15,15,15,15,15,15,15), tol = 0.01)))
models.append(('Rede Neural - MLP8 25',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(25,25,25,25,25,25,25,25), tol = 0.01)))
models.append(('Rede Neural - MLP8 30',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(30,30,30,30,30,30,30,30), tol = 0.01)))
models.append(('Rede Neural - MLP9 10',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(10,10,10,10,10,10,10,10,10), tol = 0.01)))
models.append(('Rede Neural - MLP9 15',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(15,15,15,15,15,15,15,15,15), tol = 0.01)))
models.append(('Rede Neural - MLP9 25',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(25,25,25,25,25,25,25,25,25), tol = 0.01)))
models.append(('Rede Neural - MLP9 30',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(30,30,30,30,30,30,30,30,30), tol = 0.01)))



In [67]:
a,b = models_score(models, X_train, y_train, X_test, y_test, scoring = 'neg_mean_absolute_percentage_error' , n_splits=10)

Iniciando processamento do modelo Rede Neural - MLP5 10
Total training+CV time (s): 0:06:00.205693
Iniciando processamento do modelo Rede Neural - MLP5 15
Total training+CV time (s): 0:08:10.543307
Iniciando processamento do modelo Rede Neural - MLP5 25
Total training+CV time (s): 0:22:47.217750
Iniciando processamento do modelo Rede Neural - MLP5 30
Total training+CV time (s): 0:28:21.765266
Iniciando processamento do modelo Rede Neural - MLP8 10
Total training+CV time (s): 0:06:04.937514
Iniciando processamento do modelo Rede Neural - MLP8 15
Total training+CV time (s): 0:07:35.875170
Iniciando processamento do modelo Rede Neural - MLP8 25
Total training+CV time (s): 0:31:19.906471
Iniciando processamento do modelo Rede Neural - MLP8 30
Total training+CV time (s): 0:35:09.795789
Iniciando processamento do modelo Rede Neural - MLP9 10
Total training+CV time (s): 0:07:18.625839
Iniciando processamento do modelo Rede Neural - MLP9 15
Total training+CV time (s): 0:06:15.484490
Iniciando 

,Name,CV MAPE mean,CV MAPE std,MAE (train),MAE (test),Max Error (train),Max Error (test),MAPE (train),MAPE (test),R2 (train),R2 (test),Training time (s)
0,Rede Neural - MLP9 30,-8.86,1.28,"23,479.65","27,402.12","3,822,966.93","1,513,308.99",6.87,6.92,0.63,0.32,271.27
1,Rede Neural - MLP8 30,-8.62,0.85,"24,625.68","28,174.06","3,868,140.74","1,524,081.04",7.41,7.36,0.60,0.29,318.91
2,Rede Neural - MLP9 25,-9.24,0.88,"25,000.43","27,652.30","3,842,582.30","1,800,941.72",7.83,8.02,0.55,0.29,77.48
3,Rede Neural - MLP9 15,-9.67,0.67,"27,277.08","29,400.56","3,566,126.64","1,919,458.80",9.15,9.01,0.51,0.32,36.97
4,Rede Neural - MLP8 15,-9.30,0.75,"27,199.50","28,839.94","3,696,068.51","1,987,024.68",9.15,9.07,0.50,0.31,64.32
5,Rede Neural - MLP5 30,-9.09,0.93,"26,326.23","28,646.64","3,639,101.95","1,765,779.72",8.79,9.10,0.52,0.31,154.20
6,Rede Neural - MLP5 15,-9.31,0.84,"26,804.98","28,975.57","3,574,976.37","1,892,418.77",9.19,9.28,0.48,0.27,53.26
7,Rede Neural - MLP9 10,-9.87,0.75,"26,976.62","28,305.25","3,575,089.11","2,023,240.61",9.24,9.40,0.45,0.29,32.99
8,Rede Neural - MLP5 25,-9.62,0.88,"26,659.28","28,691.97","3,821,298.10","1,763,549.62",10.03,9.90,0.49,0.26,111.15
9,Rede Neural - MLP5 10,-10.16,1.16,"27,897.39","28,976.22","3,255,008.12","1,898,419.14",10.12,10.04,0.44,0.32,44.03


### 6.1.2 Floresta aleatória

In [78]:
models=[]
models.append(('RF 03', RandomForestRegressor(random_state=0, max_depth = 3, n_jobs=4)))
models.append(('RF 06', RandomForestRegressor(random_state=0, max_depth = 6, n_jobs=4)))
models.append(('RF 09', RandomForestRegressor(random_state=0, max_depth = 9, n_jobs=4)))
models.append(('RF 12', RandomForestRegressor(random_state=0, max_depth = 12, n_jobs=4)))
models.append(('RF 15', RandomForestRegressor(random_state=0, max_depth = 15, n_jobs=4)))
models.append(('RF 18', RandomForestRegressor(random_state=0, max_depth = 18, n_jobs=4)))
models.append(('RF 21', RandomForestRegressor(random_state=0, max_depth = 21, n_jobs=4)))
models.append(('RF 24', RandomForestRegressor(random_state=0, max_depth = 24, n_jobs=4)))
models.append(('RF 27', RandomForestRegressor(random_state=0, max_depth = 27, n_jobs=4)))
models.append(('RF 30', RandomForestRegressor(random_state=0, max_depth = 30, n_jobs=4)) )
models.append(('RF 33', RandomForestRegressor(random_state=0, max_depth = 33, n_jobs=4)))
models.append(('RF 36', RandomForestRegressor(random_state=0, max_depth = 36, n_jobs=4)))

In [79]:
a,b = models_score(models, X_train, y_train, X_test, y_test, scoring = 'neg_mean_absolute_percentage_error' , n_splits=10)

Iniciando processamento do modelo RF 03
Total training+CV time (s): 0:00:09.457044
Iniciando processamento do modelo RF 06
Total training+CV time (s): 0:00:18.521148
Iniciando processamento do modelo RF 09
Total training+CV time (s): 0:00:28.161684
Iniciando processamento do modelo RF 12
Total training+CV time (s): 0:00:34.977000
Iniciando processamento do modelo RF 15
Total training+CV time (s): 0:00:43.929389
Iniciando processamento do modelo RF 18
Total training+CV time (s): 0:00:55.114749
Iniciando processamento do modelo RF 21
Total training+CV time (s): 0:01:05.042101
Iniciando processamento do modelo RF 24
Total training+CV time (s): 0:01:08.726922
Iniciando processamento do modelo RF 27
Total training+CV time (s): 0:01:20.733978
Iniciando processamento do modelo RF 30
Total training+CV time (s): 0:01:23.631197
Iniciando processamento do modelo RF 33
Total training+CV time (s): 0:01:19.132936
Iniciando processamento do modelo RF 36
Total training+CV time (s): 0:01:08.849550
----

,Name,CV MAPE mean,CV MAPE std,MAE (train),MAE (test),Max Error (train),Max Error (test),MAPE (train),MAPE (test),R2 (train),R2 (test),Training time (s)
0,RF 33,-2.13,0.24,"5,984.05","16,679.37","2,900,459.10","920,829.30",0.74,2.17,0.95,0.62,5.64
1,RF 36,-2.12,0.24,"5,970.69","16,717.09","2,966,236.38","923,907.05",0.73,2.17,0.95,0.62,7.11
2,RF 24,-2.14,0.25,"6,222.69","16,734.38","2,904,853.01","921,894.45",0.78,2.17,0.95,0.62,5.64
3,RF 30,-2.13,0.24,"5,988.74","16,670.50","2,934,466.47","920,176.45",0.74,2.17,0.95,0.62,8.20
4,RF 27,-2.13,0.25,"6,045.58","16,728.50","2,893,912.62","916,056.56",0.74,2.18,0.95,0.62,8.12
5,RF 21,-2.19,0.25,"6,781.83","16,936.65","2,954,170.80","927,685.85",0.89,2.24,0.95,0.62,7.84
6,RF 18,-2.37,0.24,"8,214.55","17,381.86","2,926,428.26","907,121.58",1.21,2.40,0.94,0.62,5.10
7,RF 15,-2.89,0.26,"11,188.99","18,788.01","2,893,923.28","935,142.57",1.99,2.89,0.92,0.60,4.51
8,RF 12,-4.01,0.26,"15,875.89","21,316.06","2,942,539.68","969,825.46",3.45,3.96,0.84,0.55,2.91
9,RF 09,-5.66,0.29,"21,360.55","24,466.11","2,917,504.62","1,367,420.66",5.43,5.49,0.70,0.49,2.45


## 6.2 Outros modelos

### 6.2.1 Regressão Linear

In [71]:
models = []
models.append(('Regressao Linear',LinearRegression()))

### 6.2.2 Árvore de decisão

In [72]:
models.append(('Árvore de Decisão',DecisionTreeRegressor(random_state=0)))

### 6.2.3 Support Vector Machine

In [73]:
models.append(('Support Vector Machine',SVR(kernel='rbf')))

### 6.2.4 Árvore Extremamente Aleatória

In [74]:
models.append(('Árvore Extramamente Aleatória',ExtraTreesRegressor(n_estimators=100, random_state=0)))

### 6.2.5 Gradiente Boosting

In [75]:
models.append(('Gradient Boosting',GradientBoostingRegressor(random_state=0, n_estimators=100)))

### 6.2.6 MLP Regressor e Random Forest

In [76]:
models.append(('Rede Neural - MLP9 30',MLPRegressor(max_iter=2000, random_state=0, hidden_layer_sizes=(30,30,30,30,30,30,30,30,30), tol = 0.01)))
models.append(('RF 33', RandomForestRegressor(random_state=0, max_depth = 33, n_jobs=4)))

## 6.2 Análise do melhor cenário

In [77]:
a,b = models_score(models, X_train, y_train, X_test, y_test, scoring = 'neg_mean_absolute_percentage_error' , n_splits=10)

Iniciando processamento do modelo Regressao Linear
Total training+CV time (s): 0:00:00.501519
Iniciando processamento do modelo Árvore de Decisão
Total training+CV time (s): 0:00:05.095200
Iniciando processamento do modelo Support Vector Machine
Total training+CV time (s): 0:28:02.365772
Iniciando processamento do modelo Árvore Extramamente Aleatória
Total training+CV time (s): 0:03:02.402164
Iniciando processamento do modelo Gradient Boosting
Total training+CV time (s): 0:01:18.485726
Iniciando processamento do modelo Rede Neural - MLP9 30
Total training+CV time (s): 0:59:22.626219
Iniciando processamento do modelo RF 33
Total training+CV time (s): 0:02:52.795479
----------------RESULTS-------------- 

Best MAPE in train (CV mean): -13.87
Best MAE in test: 16679.37
Best MAPE in test: 1.73 

Best R2 in test: 0.07 

Best Max_error in test: 0.00 

-----------MODELS COMPARISON--------- 



,Name,CV MAPE mean,CV MAPE std,MAE (train),MAE (test),Max Error (train),Max Error (test),MAPE (train),MAPE (test),R2 (train),R2 (test),Training time (s)
0,Árvore de Decisão,-1.66,0.35,0.00,"20,150.71","3,132,768.00",0.00,0.00,1.73,1.00,0.43,0.49
1,RF 33,-2.13,0.24,"5,984.05","16,679.37","2,900,459.10","920,829.30",0.74,2.17,0.95,0.62,18.68
2,Árvore Extramamente Aleatória,-2.99,0.35,0.00,"20,644.29","3,417,137.16",0.00,0.00,2.87,1.00,0.46,17.88
3,Support Vector Machine,-4.40,0.14,"31,756.54","32,117.38","4,040,170.83","2,489,284.11",4.31,4.39,-0.08,-0.07,127.88
4,Rede Neural - MLP9 30,-8.86,1.28,"23,479.65","27,402.12","3,822,966.93","1,513,308.99",6.87,6.92,0.63,0.32,383.61
5,Gradient Boosting,-8.74,0.31,"26,030.95","26,876.46","3,730,865.81","2,127,870.86",8.56,8.64,0.49,0.42,7.26
6,Regressao Linear,-13.87,0.69,"33,972.67","33,861.72","3,934,013.70","2,426,408.14",13.85,13.42,0.16,0.16,0.02


# Construção do modelo escolhido

In [54]:
random_forest = RandomForestRegressor(random_state=0, max_depth = 33, n_jobs=4)
random_forest.fit(X_train,y_train)